<a href="https://colab.research.google.com/github/josvaldes/trabajoGradoMCD/blob/PrimeraBD/scrapingColombiaTic_final2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# ============================================================
# === 1. Montar Google Drive y preparar entorno ===============
# ============================================================

from google.colab import drive
import os, requests, shutil, re, pandas as pd, traceback
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from datetime import datetime
import duckdb

# --- Montar Google Drive ---
drive.mount('/content/gdrive')
base_path = "/content/gdrive/MyDrive/trabajoGrado/Codigo/reporteColombiaTic"

# --- Crear carpetas necesarias ---
temp_folder = os.path.join(base_path, "temp_descargas")
final_folder = os.path.join(base_path, "reportes")
os.makedirs(temp_folder, exist_ok=True)
os.makedirs(final_folder, exist_ok=True)

print(f"✅ Google Drive detectado en: /content/gdrive")
print(f"📂 Carpeta base lista: {base_path}")

# ============================================================
# === 2. Funciones auxiliares ================================
# ============================================================

log_file = os.path.join(base_path, "descargas_log.csv")

def registrar_log(nombre, url, size, carpeta, estado, fallo=""):
    """Registrar proceso de descarga en log CSV."""
    registro = pd.DataFrame([{
        "archivo": nombre,
        "url": url,
        "tamano_mb": size,
        "ubicacion": carpeta,
        "estado": estado,
        "detalle": fallo,
        "fecha": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }])
    if os.path.exists(log_file):
        registro.to_csv(log_file, mode="a", header=False, index=False)
    else:
        registro.to_csv(log_file, index=False)
    return registro

def limpiar_nombre(texto):
    texto = texto.lower()
    texto = re.sub(r"[^a-z0-9áéíóúñ ]", "", texto)
    texto = texto.replace(" ", "_")
    return texto.strip("_")

# ============================================================
# === 3. Conectarse al portal ColombiaTIC ====================
# ============================================================

base_url = "https://colombiatic.mintic.gov.co/679/w3-channel.html"
print(f"\n🌐 Accediendo al portal: {base_url}")

try:
    resp = requests.get(base_url, timeout=20)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "lxml")

    boletin_links = [
        (a.text.strip(), urljoin(base_url, a["href"]))
        for a in soup.find_all("a", href=True)
        if "w3-article" in a["href"]
    ]

    print(f"📰 Se encontraron {len(boletin_links)} boletines publicados.")
except Exception as e:
    print(f"⚠️ No se pudo acceder al portal ({e}).")
    boletin_links = []

# ============================================================
# === 4. Descargar archivos relevantes =======================
# ============================================================

print("\n📡 Analizando boletines...")

for titulo, boletin_url in boletin_links:
    # Solo boletines del sector TIC
    if "tic" not in titulo.lower():
        continue

    try:
        print(f"\n🔎 Revisando boletín: {titulo}")
        res = requests.get(boletin_url, timeout=20)
        res.raise_for_status()
        s = BeautifulSoup(res.text, "lxml")

        for a in s.find_all("a", href=True):
            href = a["href"]
            if href.lower().endswith((".xlsx", ".xls")):
                file_url = urljoin(boletin_url, href)
                nombre_archivo = limpiar_nombre(titulo) + ".xlsx"
                ruta_temp = os.path.join(temp_folder, nombre_archivo)

                # Evitar duplicados
                if os.path.exists(os.path.join(final_folder, nombre_archivo)):
                    print(f"⏭️ Ya existe: {nombre_archivo}, se omite descarga.")
                    continue

                # Tamaño del archivo
                head = requests.head(file_url, timeout=20)
                size_mb = int(head.headers.get("Content-Length", 0)) / (1024 * 1024)

                if size_mb > 150:
                    print(f"⏭️ Saltando {nombre_archivo} (archivo grande: {size_mb:.1f} MB)")
                    registrar_log(nombre_archivo, file_url, size_mb, "temp_descargas", "omitido")
                    continue

                print(f"⬇️ Descargando: {nombre_archivo} ({size_mb:.1f} MB)")
                r = requests.get(file_url, timeout=60)
                r.raise_for_status()
                with open(ruta_temp, "wb") as f:
                    f.write(r.content)
                print(f"✅ Guardado temporalmente: {ruta_temp}")
                registrar_log(nombre_archivo, file_url, size_mb, "temp_descargas", "descargado")

    except Exception as e:
        print(f"⚠️ Error en boletín {titulo}: {e}")
        registrar_log(titulo, boletin_url, 0, "temp_descargas", "error", f"{e}")

# ============================================================
# === 5. Mover archivos válidos a carpeta final ===============
# ============================================================

print("\n🔍 Validando archivos descargados...")
archivos_temp = [f for f in os.listdir(temp_folder) if f.lower().endswith((".xlsx", ".xls"))]
print(f"📦 Total archivos temporales: {len(archivos_temp)}")

for archivo in archivos_temp:
    shutil.move(os.path.join(temp_folder, archivo), os.path.join(final_folder, archivo))
    registrar_log(archivo, "-", 0, "reportes", "movido_a_final")

# Eliminar carpeta temporal si está vacía
if not os.listdir(temp_folder):
    os.rmdir(temp_folder)
    print("🧺 Carpeta temporal vacía eliminada.")

print(f"\n📁 Carpeta final lista para procesamiento: {final_folder}")
print(f"🧾 Log actualizado: {log_file}")

# ============================================================
# === 6. Cargar archivos Excel en DuckDB (resiliente) =========
# ============================================================

db_path = "/content/gdrive/MyDrive/trabajoGrado/reporteColombiaTic.db"
backup_path = "/content/gdrive/MyDrive/trabajoGrado/reporteColombiaTic_backup.db"

def conectar_duckdb_seguro(ruta):
    """Intenta conectar a DuckDB y reconstruir si la base está dañada."""
    try:
        con = duckdb.connect(ruta)
        con.execute("PRAGMA version;").fetchall()
        print(f"✅ Base verificada: {ruta}")
        return con
    except Exception as e:
        print(f"⚠️ Error al abrir la base ({e}). Intentando reparar...")
        if os.path.exists(ruta):
            shutil.move(ruta, backup_path)
            if os.path.exists(ruta + ".wal"):
                os.remove(ruta + ".wal")
            print(f"🗂️ Copia de seguridad creada: {backup_path}")
        con = duckdb.connect(ruta)
        print(f"🆕 Nueva base creada en: {ruta}")
        return con

con = conectar_duckdb_seguro(db_path)

# 🔹 Crear una secuencia (contador automático)
con.execute("CREATE SEQUENCE IF NOT EXISTS seq_control START 1;")

# 🔹 Crear tabla con ID autogenerado usando la secuencia
con.execute("""
CREATE TABLE IF NOT EXISTS control_cargue (
    id BIGINT PRIMARY KEY DEFAULT nextval('seq_control'),
    archivo VARCHAR,
    hoja VARCHAR,
    columnas_detectadas VARCHAR,
    filas INTEGER,
    fecha_cargue TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
""")

# ============================================================
# === Reparar tabla control_cargue si tiene definición vieja ==
# ============================================================

try:
    tabla_info = con.execute("PRAGMA table_info('control_cargue')").df()
    if 'id' not in tabla_info['name'].tolist():
        raise Exception("Tabla control_cargue inválida (sin campo id).")

    # Verificar si la columna id tiene un default autoincremental
    if not any("nextval" in str(d) for d in tabla_info['dflt_value']):
        print("⚠️ Tabla control_cargue sin secuencia de autoincremento. Corrigiendo...")

        # Respaldar datos antiguos si existen
        try:
            df_old = con.execute("SELECT * FROM control_cargue").df()
        except:
            df_old = pd.DataFrame()

        # Eliminar tabla vieja
        con.execute("DROP TABLE IF EXISTS control_cargue;")
        con.execute("DROP SEQUENCE IF EXISTS seq_control;")

        # Crear secuencia e tabla nuevas
        con.execute("CREATE SEQUENCE IF NOT EXISTS seq_control START 1;")
        con.execute("""
        CREATE TABLE control_cargue (
            id BIGINT PRIMARY KEY DEFAULT nextval('seq_control'),
            archivo VARCHAR,
            hoja VARCHAR,
            columnas_detectadas VARCHAR,
            filas INTEGER,
            fecha_cargue TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        """)

        # Restaurar datos antiguos (si existían)
        if not df_old.empty:
            con.append("control_cargue", df_old)
            print(f"✅ Tabla control_cargue reparada y restaurados {len(df_old)} registros.")
        else:
            print("✅ Tabla control_cargue creada desde cero (no había datos previos).")

    else:
        print("✅ Tabla control_cargue ya tiene secuencia de autoincremento.")
except Exception as e:
    print(f"⚠️ No existía tabla control_cargue, creando nueva... ({e})")

    con.execute("DROP SEQUENCE IF EXISTS seq_control;")
    con.execute("CREATE SEQUENCE IF NOT EXISTS seq_control START 1;")
    con.execute("""
    CREATE TABLE control_cargue (
        id BIGINT PRIMARY KEY DEFAULT nextval('seq_control'),
        archivo VARCHAR,
        hoja VARCHAR,
        columnas_detectadas VARCHAR,
        filas INTEGER,
        fecha_cargue TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    """)
    print("✅ Tabla control_cargue creada correctamente.")



# --- Cargar solo archivos nuevos ---
archivos_finales = [f for f in os.listdir(final_folder) if f.lower().endswith((".xlsx", ".xls"))]
print(f"\n📦 Archivos Excel disponibles: {len(archivos_finales)}")

try:
    ya_cargados = set(con.execute("SELECT DISTINCT archivo FROM control_cargue").fetchdf()["archivo"].tolist())
except:
    ya_cargados = set()

archivos_nuevos = [f for f in archivos_finales if f not in ya_cargados]

if not archivos_nuevos:
    print("⏭️ No hay archivos nuevos para cargar. La base está actualizada.")
else:
    print(f"🆕 Archivos nuevos detectados: {len(archivos_nuevos)}")
    for nombre_archivo in archivos_nuevos:
        ruta_archivo = os.path.join(final_folder, nombre_archivo)
        print(f"\n📘 Procesando archivo nuevo: {nombre_archivo}")

        try:
            xls = pd.ExcelFile(ruta_archivo)
            hojas_excel = xls.sheet_names
            print(f"   🧾 Hojas detectadas: {hojas_excel[:10]}...")

            for hoja in hojas_excel:
                for fila_encabezado in range(0, 10):
                    df = pd.read_excel(ruta_archivo, sheet_name=hoja, header=fila_encabezado)
                    if df.columns.notna().sum() > 2:
                        break

                nombre_archivo_base = os.path.splitext(nombre_archivo)[0].lower()
                nombre_archivo_base = re.sub(r"[^a-z0-9_]", "", nombre_archivo_base)
                nombre_hoja_normalizada = re.sub(r"[^a-z0-9_]", "", hoja.lower())
                nombre_tabla = f"{nombre_archivo_base}_{nombre_hoja_normalizada}"

                print(f"   📊 Cargando hoja '{hoja}' como tabla '{nombre_tabla}' ({len(df)} filas)")
                # ✅ Convertir todas las columnas a texto para evitar errores de tipo
                df = df.astype(str)

                # Crear la tabla en DuckDB
                try:
                    con.execute(f"CREATE OR REPLACE TABLE '{nombre_tabla}' AS SELECT * FROM df")
                    print(f"   ✅ Tabla creada: {nombre_tabla} ({len(df)} filas)")

                    # Registrar en control_cargue
                    con.execute("""
                        INSERT INTO control_cargue (archivo, hoja, columnas_detectadas, filas)
                        VALUES (?, ?, ?, ?)
                    """, [nombre_archivo, hoja, ", ".join(map(str, df.columns)), len(df)])

                except Exception as e:
                    print(f"   ⚠️ Error al crear tabla {nombre_tabla}: {e}")

                # ✅ Insert corregido: deja que DuckDB genere automáticamente el ID
                con.execute("""
                    INSERT INTO control_cargue (archivo, hoja, columnas_detectadas, filas)
                    VALUES (?, ?, ?, ?)
                """, [nombre_archivo, hoja, ", ".join(map(str, df.columns)), len(df)])

            print(f"✅ Archivo '{nombre_archivo}' cargado completamente.")

        except Exception as e:
            print(f"⚠️ Error procesando {nombre_archivo}: {e}")
            traceback.print_exc()

print("\n📊 Tablas actuales en la base:")
print(con.execute("SHOW TABLES").df())

print("\n📋 Estado de control_cargue:")
print(con.execute("""
    SELECT archivo, COUNT(*) hojas_cargadas, SUM(filas) filas_totales
    FROM control_cargue
    GROUP BY archivo
    ORDER BY archivo
""").df())

con.close()
print("\n🎯 Base validada, actualizada y lista para análisis.")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
✅ Google Drive detectado en: /content/gdrive
📂 Carpeta base lista: /content/gdrive/MyDrive/trabajoGrado/Codigo/reporteColombiaTic

🌐 Accediendo al portal: https://colombiatic.mintic.gov.co/679/w3-channel.html
📰 Se encontraron 5 boletines publicados.

📡 Analizando boletines...

🔎 Revisando boletín: BoletÃ­n trimestral del sector TIC - Cifras primer trimestre de 2025
⏭️ Ya existe: boletn_trimestral_del_sector_tic__cifras_primer_trimestre_de_2025.xlsx, se omite descarga.
⏭️ Ya existe: boletn_trimestral_del_sector_tic__cifras_primer_trimestre_de_2025.xlsx, se omite descarga.

🔎 Revisando boletín: PolÃ­tica de privacidad y condiciones de uso

🔎 Revisando boletín: Mintic y la CertificaciÃ³n ISO

🔍 Validando archivos descargados...
📦 Total archivos temporales: 0
🧺 Carpeta temporal vacía eliminada.

📁 Carpeta final lista para procesamiento: /content/gdrive/MyDrive/

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

   ✅ Tabla creada: reporte_tic_cuarto_trimestre_2024_41 (571379 filas)
   📊 Cargando hoja '4,2' como tabla 'reporte_tic_cuarto_trimestre_2024_42' (816983 filas)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

   ✅ Tabla creada: reporte_tic_cuarto_trimestre_2024_42 (816983 filas)
   📊 Cargando hoja '4,3' como tabla 'reporte_tic_cuarto_trimestre_2024_43' (763700 filas)


...

   ✅ Tabla creada: reporte_tic_segundo_trimestre_20241_43 (812686 filas)
   📊 Cargando hoja '4,4' como tabla 'reporte_tic_segundo_trimestre_20241_44' (383654 filas)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

   ✅ Tabla creada: reporte_tic_segundo_trimestre_20241_44 (383654 filas)
   📊 Cargando hoja '5' como tabla 'reporte_tic_segundo_trimestre_20241_5' (12132 filas)
   ✅ Tabla creada: reporte_tic_segundo_trimestre_20241_5 (12132 filas)
   📊 Cargando hoja '6' como tabla 'reporte_tic_segundo_trimestre_20241_6' (594 filas)
   ✅ Tabla creada: reporte_tic_segundo_trimestre_20241_6 (594 filas)
   📊 Cargando hoja '7' como tabla 'reporte_tic_segundo_trimestre_20241_7' (242 filas)
   ✅ Tabla creada: reporte_tic_segundo_trimestre_20241_7 (242 filas)
   📊 Cargando hoja '8' como tabla 'reporte_tic_segundo_trimestre_20241_8' (172 filas)
   ✅ Tabla creada: reporte_tic_segundo_trimestre_20241_8 (172 filas)
   📊 Cargando hoja '9' como tabla 'reporte_tic_segundo_trimestre_20241_9' (577 filas)
   ✅ Tabla creada: reporte_tic_segundo_trimestre_20241_9 (577 filas)
   📊 Cargando hoja '10' como tabla 'reporte_tic_segundo_trimestre_20241_10' (246 filas)
   ✅ Tabla creada: reporte_tic_segundo_trimestre_20241_10 (2

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parsers/python_parser.py", line 716, in _next_line
    line = self._check_comments([self.data[self.pos]])[0]
                                 ~~~~~~~~~^^^^^^^^^^
IndexError: list index out of range

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parsers/python_parser.py", line 407, in _infer_columns
    line = self._next_line()
           ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parsers/python_parser.py", line 729, in _next_line
    raise StopIteration
StopIteration

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/tmp/ipython-input-819696096.py", line 279, in <cell line: 0>
    df = pd.read_excel(ruta_archivo, sheet_name=hoja, header=fila_encabezado)
         ^^^^^^^^^^^^^^^^^^^^^^^


🎯 Base validada, actualizada y lista para análisis.


In [19]:
con = conectar_duckdb_seguro(db_path)

print(con.execute("SELECT * FROM control_cargue ORDER BY fecha_cargue DESC").df())

con.close()


✅ Base verificada: /content/gdrive/MyDrive/trabajoGrado/reporteColombiaTic.db
      id                                            archivo       hoja  \
0    132        reporte_tic_segundo_trimestre_2024 (1).xlsx         17   
1    131        reporte_tic_segundo_trimestre_2024 (1).xlsx         17   
2    130        reporte_tic_segundo_trimestre_2024 (1).xlsx         16   
3    129        reporte_tic_segundo_trimestre_2024 (1).xlsx         16   
4    128        reporte_tic_segundo_trimestre_2024 (1).xlsx         15   
..   ...                                                ...        ...   
127    5             reporte_tic_cuarto_trimestre_2024.xlsx  CONTENIDO   
128    4             reporte_tic_cuarto_trimestre_2024.xlsx    PORTADA   
129    3             reporte_tic_cuarto_trimestre_2024.xlsx    PORTADA   
130    2  boletn_trimestral_del_sector_tic__cifras_prime...  CONTENIDO   
131    1  boletn_trimestral_del_sector_tic__cifras_prime...    PORTADA   

                                 